In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris

In [2]:
iris_dataset = load_iris()
df = pd.DataFrame(iris_dataset["data"], columns=iris_dataset.feature_names)
df["target"] = iris_dataset["target"]
print(df)

     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                  5.1               3.5                1.4               0.2   
1                  4.9               3.0                1.4               0.2   
2                  4.7               3.2                1.3               0.2   
3                  4.6               3.1                1.5               0.2   
4                  5.0               3.6                1.4               0.2   
..                 ...               ...                ...               ...   
145                6.7               3.0                5.2               2.3   
146                6.3               2.5                5.0               1.9   
147                6.5               3.0                5.2               2.0   
148                6.2               3.4                5.4               2.3   
149                5.9               3.0                5.1               1.8   

     target  
0         0  

In [66]:
def split_df_train(df: pd.DataFrame, ratio=0.2):
    df_shuffled = df.sample(frac=1).reset_index(drop=True)
    split_index = int(len(df_shuffled) * (1 - ratio))
    df_train = df_shuffled.iloc[:split_index]
    df_test = df_shuffled.iloc[split_index:]
    x_train = df_train.drop(columns=['target'])
    y_train=df_train['target']
    x_test = df_test.drop(columns=['target'])
    y_test = df_test['target']
    return x_train, y_train, x_test, y_test

class knn_classifier:
    def __init__(self, k:int = 3):
        self.k = k
        self.x_train = None
        self.y_train = None
    
    def fit(self, x_train, y_train):
        self.x_train = x_train.values
        self.y_train = y_train.values
        return self
    
    def calc_distance(self, point1: np.ndarray, point2: np.ndarray):
        return np.sqrt(np.sum((point1 - point2) ** 2))

    def calc_majority(self, targets):
        return max(set(targets), key=targets.count)
    
    def calculate_k_nearest(self, test_point):
        distances = [self.calc_distance(test_point, train_point) for train_point in self.x_train]
        k_nearest_ind = np.argsort(distances)[:self.k]
        k_nearest_targets = [self.y_train[index] for index in k_nearest_ind]
        return k_nearest_targets

    def predict(self, x: pd.DataFrame):
        predictions = []
        x = x.values

        for test_point in x:
            targets = self.calculate_k_nearest(test_point=test_point)
            target_prediction = self.calc_majority(targets=targets)
            predictions.append(target_prediction)
        return predictions



    
    

In [67]:
x_train, y_train, x_test, y_test = split_df_train(df=df)
knn = knn_classifier()
knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)
accuracy = np.mean(y_pred==y_test)
